# Magnetic Field Calculations

M&I Ch 17 Magnetic Field

- §17.3  &ndash; Biot-Savart Law: Single Moving Charge
- §17.7  &ndash; The Magnetic Field of Current Distributions
- §17.8  &ndash; A Circular Loop of Wire
- §17.11 &ndash; The Magnetic Field of a Bar Magnet

In [1]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>
>   
>
>   Conventional current flows in a ring in the direction indicated in the 
>   diagram (if you stand at a location $A$ on the $+x$-axis, and look toward 
>   the ring, current flows clockwise).
>
>   __At each of the locations labeled by a letter, use the right hand rule to find the direction of the magnetic field at that location, due to the current in the ring.__
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='1a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Location A__ (select one:)
>
>   - [ ] &nbsp; None
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Location B__ (select one:)
>
>   - [ ] &nbsp; None
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$ 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Location C__ (select one:)
>
>   - [ ] &nbsp; None
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(D)
>
>   __Location D__ (select one:)
>
>   - [ ] &nbsp; None
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>   
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(E)
>
>   __Location E__ (select one:)
>
>   - [ ] &nbsp; None
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B9;&hairsp;&#x2E05;&hairsp;(F)
>
>   __Location F__ (select one:)
>
>   - [ ] &nbsp; None
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]
>
>   A vector $\vec{C}$ of magnitude $4$ lies along the $x$-axis, and a vector 
>   $\vec{D}$ of magnitude $8$ lies in the $xy$-plane, 20 degrees from the 
>   $x$-axis (see the figure below).
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='2a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the magnitude of the cross product $\vec{C} \times \vec{D}$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert \vec{C} \times \vec{D}\rvert = \fbox{\hspace{8em}?}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='2b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the direction of the cross product $\vec{C} \times \vec{D}$?__ 
>
>   (select one:)
>
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; in the $xz$-plane, but not along the $x$ or $z$ axes.
>   - [ ] &nbsp; in the $xy$-plane, but not along the $x$ or $y$ axes.
>   - [ ] &nbsp; in the $yz$-plane, but not along the $x$ or $y$ axes.
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='1c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00B2;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the cross product $\vec{D} \times \vec{C}$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{D} \times \vec{C} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   If $\vec{v} = \left\langle 444, -345, 405 \right\rangle$ [m/s] and $\vec{r} = \left\langle 0.577, 0.577, -0.577 \right\rangle$ [m], __what is $\vec{v} \times \vec{r}$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{v} \times \vec{r} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle \left[\textrm{m}^2/\textrm{s}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]
>
>   __What is $\langle 4, 0, -4 \rangle \times \langle -6, 3, 5 \rangle$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]*</h2>

>   ---
>   ## [5]*
>
>   
>
>   A wire, through which a current is flowing, lies on the $x$-axis as shown in
>   the figure above. Connecting wires (which are not shown in the diagram) 
>   connect the ends of the wire to batteries (which are also not shown). 
>   Electron current flows through the wire in the $-x$ direction, as indicated 
>   in the diagram. 
>
>   To calculate the magnetic field at location $A$ due to the current in the 
>   wire, we divide the wire into pieces, approximate each piece as a point 
>   charge moving in the direction of conventional current, and calculate the 
>   magnetic field at the observation location due only to this piece; then sum 
>   the contributions of all pieces to get the net magnetic field. 
>
>   The wire is $1.6$ [m] long, and is divided into 8 pieces.  The observation 
>   location $A$ is located at $\langle 0.1, 0.22, 0\rangle$ [m].
>   The conventional current running through the wire is $7.6$ amperes. 
>
>   __In this exercise, you will calculate the magnetic field at the observation location, due only to segment 2 of the wire.__
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='5a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the direction of conventional current in this wire?__ 
>
>   (select one):
>
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>   - [ ] &nbsp; $-z$
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $+z$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(B)
>
>   __How long is segment 2?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the magnitude of the vector $\Delta \vec{l}$ for segment 2?__
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert \Delta \vec{l}\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the vector $\Delta \vec{l}$ for segment 2?__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta \vec{l} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle\ \left[\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the location of the center of segment 2?__
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{source location} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle\ \left[\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(F)
>
>   __What is the vector $\vec{r}$, from source to observation location, for segment 2?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{r} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle\ \left[\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5g' style="display:none">(G)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(G)
>
>   __What is the unit vector $\hat{r}$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\hat{r} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5h' style="display:none">(H)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(H)
>
>   __What is $\Delta \vec{l} \times \hat{r}$?__
>
>   &emsp;&emsp;&emsp;&emsp; $\delta \vec{l} \times \hat{r} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle \left[\textrm{m}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5i' style="display:none">(I)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(I)
>
>   __Calculate the magnetic field $\Delta \mathbf{\vec{B}}$ at the observation location due only to the current in segment 2 of the wire.__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta \mathbf{\vec{B}} = \Big\langle\ \fbox{\hspace{4em}?}\ ,\ \fbox{\hspace{4em}?}\ , \fbox{\hspace{4em}?}\ \Big\rangle \left[\textrm{T}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]*</h2>

>   ---
>   ## [6]*
>
>   A compass is placed inside a triangular coil of wire with 3 turns, as shown 
>   in the diagram.
>
>   
>
>   Each side of the triangle has a length $L$.
>
>   The compass is a perpendicular distance $d$ from the center of each side of 
>   the triangle.
>   The coil is in the $xy$-plane, magnetic North (due to the Earth) is in the 
>   negative $x$ direction.
>   Conventional current runs in the coil as shown (clockwise, as viewed from a 
>   location on the $+z$-axis).
>
>   The compass needle rotates in the $xz$-plane, as indicated in the diagram. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   __While current is running through the coil, what direction does the compass needle point?__ (select one:)
>
>   - [ ] &nbsp; North
>
>   - [ ] &nbsp; North-East
>
>   - [ ] &nbsp; East
>
>   - [ ] &nbsp; SouthEast
>
>   - [ ] &nbsp; South
>
>   - [ ] &nbsp; SouthWest
>
>   - [ ] &nbsp; West
>
>   - [ ] &nbsp; NorthWest
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] &nbsp; North-East

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Which of the following statements about how to model the coil are reasonable?__ 
>
>   (check all that apply:)
>
>   - [ ] &nbsp; A circular loop is a poor model for one turn of the coil, 
>                because the distance from the wire to the compass varies so much.
>
>   - [ ] &nbsp; One turn of the coil could be modeled as 3 short straight wires.
>
>   - [ ] &nbsp; Since the coil is made from a single wire, we could model one 
>                turn of the coil as one long wire. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] (i)
- [x] (ii)

<h3 id='6c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(C)
>
>   A current of $0.65$ amperes runs through the wires. If $L = 8.2$ [cm], and 
>   $d = 2.36$ [cm], __what is the magnitude of the magnetic field at the location of the compass, due to the coil?__
>   (Remember that the coil has three turns.)
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert \mathbf{\vec{B}} \rvert = \fbox{\hspace{8em}?}\ \left[\textrm{T}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [7]:
I = 0.65
L = 8.2e-2
d = 2.36e-2
N = 3

# this is a solution to finite-difference Biot-Savart for a finite straight wire
# see §17.7
B = 3 * N * (μ_0 / (4*π)) * ((L*I) / (d*sqrt(d^2 + (L/2)^2)))

print(n(B))

B


0.0000429666560646387


107.416640175779*μ_0/π

<h3 id='6d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the approximate magnitude of the compass deflection, in degrees?__
>
>   Assume the magnitude of the horizontal component of the Earth's magnetic 
>   field is about $2 \times 10^{-5}$ [T].
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{deflection} = \fbox{\hspace{8em}?}\ \left[\textrm{degrees}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [11]:
B_EarthNorth = 2e-5

n(arctan(B / B_EarthNorth) * (180 / π))


65.0390944212714

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]
>
>   A thin wire is part of a complete electrical circuit which carries a current 
>   $I$. For this problem, consider only the piece of wire of length $d$ as shown
>   in the figure below. 
>
>   Answer the following questions based on this figure. 
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What are the magnitude and direction of the magnetic field due to the wire at $Q$ (location $\big\langle -w, 0, 0 \big\rangle$)?__
>
>   &emsp;&emsp;&emsp;&emsp; $\textrm{deflection} = \fbox{\hspace{8em}?}\ \left[\textrm{T}\right]$
>
>   (select one:)
>
>   - [ ] &nbsp; no field present
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] no field present

<h3 id='7b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the direction of the magnetic field at location $P$?__ (select one:)
>
>   - [ ] &nbsp; $+z$ direction
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] $+z$ direction

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]*
>
>   **Magnetic fields in the home**
>
>   At one time, concern was raised about the possible health effects of the 
>   small alternating (60 [Hz]) magnetic fields created by electric currents, in
>   houses and near power lines.
>   
>   In a house, most wires carry a maximum of $15$ amperes (there are $15$ [A] 
>   fuses that melt and break the circuit, if this current is exceeded).
>
>   The two wires in a home power cord are about $s = 3$ millimeters apart, as 
>   shown in the figure, and at any instant they carry currents in opposite 
>   directions (both of which change direction 60 times per second).
>
>   
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='8a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Calculate the maximum magnitude of the alternating magnetic field__, 
>   $d = 45$ [cm] away from the center of a long straight power cord that 
>   carries a current of $15$ amperes. 
>   Both wires are at the same height as the observation location. 
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\textrm{T}\right]$
>
>   (The magnitude of the field that you calculate is very small compared to the
>   Earth's magnetic field, but there were questions as to whether a very small 
>   alternating magnetic field might have health effects.
>   After many detailed studies, the consensus of most scientists now seems to 
>   be that these small alternating magnetic fields are not a hazard after all.)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='8b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2078;&hairsp;&#x2E05;&hairsp;(B)
>
>   Twisting the pair of wires into a braid as shown would minimize the magnetic
>   field at the location discussed in Part (A). 
>
>   __Do you see why?__
>
>   
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]*
>
>   A bar magnet is aligned East-West, with its center $38$ [cm] from the center
>   of a compass (see figure below). 
>   The compass is observed to deflect $57^\circ$ away from North as shown, and 
>   the horizontal component of the Earth's magnetic field is known to be $2 \times 10^{-5}$ [T]. 
>
>   
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='9a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Label the N and S poles of the bar magnet and explain your choice.__ 
>
>   (select one:)
>
>   - [ ] &nbsp; The N pole is on the **right** and the S pole is on the **left**, 
>                because the N pole of the compass needle points toward magnetic 
>                **north**.
>
>   - [ ] &nbsp; The N pole is on the **left** and the S pole is on the **right**,
>                because the N pole of the compass needle points toward magnetic 
>                **south**.
>
>   - [ ] &nbsp; The N pole is on the **right** and the S pole is on the **left**, 
>                because the N pole of the compass needle points toward magnetic 
>                **south**.
>
>   - [ ] &nbsp; The N pole is on the **left** and the S pole is on the **right**, 
>                because the N pole of the compass needle points toward magnetic
>                **north**.
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Determine the magnetic dipole moment of this bar magnet, including correct units.__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}$
>
>   (select one:)
>
>   - [ ] &nbsp; $\left[\textrm{A}\cdot\textrm{m}^2\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

- [x] &nbsp; $\left[\textrm{A}\cdot\textrm{m}^2\right]$

<div style="page-break-after: always;"/> 
<h2 id='10' style="display:none">[10]</h2>

>   ---
>   ## [10]
>
>   What is the direction of the magnetic field, at the indicated locations 
>   (inside and outside), this current-carrying rectangular coil of wire shown 
>   in the figure? (Direction of conventional current is shown.)
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='10a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(A)
>
>   __At the center of the loop__ (select one:)
>
>   - [ ] &nbsp; It depends on the distance to the wires, could be zero or could be non-zero. 
>
>   - [ ] &nbsp; The magnetic field points into the page. 
>
>   - [ ] &nbsp; The magnetic field points out of the page.
>
>   - [ ] &nbsp; The magnetic field is zero.
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='10b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B9;&#x2070;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Just outside the loop__ (select one:)
>
>   - [ ] &nbsp; It depends on the distance to the wires, could be zero or could be non-zero. 
>
>   - [ ] &nbsp; The magnetic field points into the page. 
>
>   - [ ] &nbsp; The magnetic field points out of the page.
>
>   - [ ] &nbsp; The magnetic field is zero.
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

---